In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn import ensemble

In [78]:
!pip install xgboost

In [79]:
import xgboost as xgb

In [159]:
from sklearn.model_selection import GridSearchCV

In [87]:
energy_df = pd.read_csv('Combined_Data_for_model_final_2.csv')

In [88]:
energy_df.head()

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0


In [89]:
year_list = []
month_list = []
for time in energy_df['Date']:
    year = str(time)[:4]
    month = str(time)[5:7]
    year_list.append(year)
    month_list.append(month)

In [90]:
energy_df['month'] = [int(m) for m in month_list]

In [111]:
energy_df['minute_interval'] = [int(energy_df['Date'][i][14:16]) for i in range(len(energy_df['Date']))]

In [112]:
energy_df

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,minute_interval,Season,Hour of day,Day of Week,Day
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,15,0,0,4,01
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,30,0,0,4,01
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,45,0,0,4,01
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,0,1,4,01
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,15,0,1,4,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,210426,2021-12-31 22:45:00,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,45,0,22,4,31
210427,210427,2021-12-31 23:00:00,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,0,23,4,31
210428,210428,2021-12-31 23:15:00,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,15,0,23,4,31
210429,210429,2021-12-31 23:30:00,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,30,0,23,4,31


In [96]:
months_unique = set(month_list)
months_unique

{'01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'}

In [97]:
energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210431 entries, 0 to 210430
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         210431 non-null  int64  
 1   Date               210431 non-null  object 
 2   Haryana Load (MW)  210431 non-null  float64
 3   temp               210431 non-null  float64
 4   dew                210431 non-null  float64
 5   humidity           210431 non-null  float64
 6   windspeed          210431 non-null  float64
 7   winddir            210431 non-null  float64
 8   cloudcover         210431 non-null  float64
 9   visibility         210431 non-null  float64
 10  month              210431 non-null  int64  
 11  minute_interval    210431 non-null  object 
dtypes: float64(8), int64(2), object(2)
memory usage: 19.3+ MB


In [98]:
def get_season(TimeSeries):
    
    season_list = []
    
    year_list = []
    month_list = []
    for time in TimeSeries:
        year = str(time)[:4]
        month = str(time)[5:7]
        year_list.append(year)
        month_list.append(month) 
       
    for m in month_list:
        if m=='01' or m=='02' or m=='12':
            season_list.append(0)   # Winter
        elif m=='03' or m=='04' or m=='05' or m=='06':
            season_list.append(1) #Spring-Summer
        elif m=='07' or m=='08' or m=='09':
            season_list.append(2) #Monsoon
        elif m=='10' or m=='11':
            season_list.append(3) #Autumn
            
    #season = pd.Series(season_list)
    return season_list

In [99]:
season = get_season(energy_df['Date'])

In [100]:
energy_df['Season'] = season

In [101]:
hour_list = []
for time in energy_df['Date']:
    hour = str(time)[11:13]
    hour_list.append(hour)
hour_list

['00',
 '00',
 '00',
 '01',
 '01',
 '01',
 '01',
 '02',
 '02',
 '02',
 '02',
 '03',
 '03',
 '03',
 '03',
 '04',
 '04',
 '04',
 '04',
 '05',
 '05',
 '05',
 '05',
 '06',
 '06',
 '06',
 '06',
 '07',
 '07',
 '07',
 '07',
 '08',
 '08',
 '08',
 '08',
 '09',
 '09',
 '09',
 '09',
 '10',
 '10',
 '10',
 '10',
 '11',
 '11',
 '11',
 '11',
 '12',
 '12',
 '12',
 '12',
 '13',
 '13',
 '13',
 '13',
 '14',
 '14',
 '14',
 '14',
 '15',
 '15',
 '15',
 '15',
 '16',
 '16',
 '16',
 '16',
 '17',
 '17',
 '17',
 '17',
 '18',
 '18',
 '18',
 '18',
 '19',
 '19',
 '19',
 '19',
 '20',
 '20',
 '20',
 '20',
 '21',
 '21',
 '21',
 '21',
 '22',
 '22',
 '22',
 '22',
 '23',
 '23',
 '23',
 '23',
 '00',
 '00',
 '00',
 '00',
 '01',
 '01',
 '01',
 '01',
 '02',
 '02',
 '02',
 '02',
 '03',
 '03',
 '03',
 '03',
 '04',
 '04',
 '04',
 '04',
 '05',
 '05',
 '05',
 '05',
 '06',
 '06',
 '06',
 '06',
 '07',
 '07',
 '07',
 '07',
 '08',
 '08',
 '08',
 '08',
 '09',
 '09',
 '09',
 '09',
 '10',
 '10',
 '10',
 '10',
 '11',
 '11',
 '11',
 '11',

In [102]:
energy_df['Hour of day'] = [int(hour) for hour in hour_list]

In [103]:
energy_df

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,minute_interval,Season,Hour of day
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,15,0,0
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,30,0,0
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,45,0,0
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,00,0,1
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,15,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,210426,2021-12-31 22:45:00,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,45,0,22
210427,210427,2021-12-31 23:00:00,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,00,0,23
210428,210428,2021-12-31 23:15:00,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,15,0,23
210429,210429,2021-12-31 23:30:00,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,30,0,23


In [108]:
energy_df['Date'] = energy_df['Date'].astype('str')

In [114]:
energy_df['Day'] = [int(energy_df['Date'][i][8:10]) for i in range(len(energy_df['Date']))]
energy_df

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,minute_interval,Season,Hour of day,Day of Week,Day
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,15,0,0,4,1
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,30,0,0,4,1
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,45,0,0,4,1
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,0,1,4,1
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,15,0,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,210426,2021-12-31 22:45:00,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,45,0,22,4,31
210427,210427,2021-12-31 23:00:00,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,0,23,4,31
210428,210428,2021-12-31 23:15:00,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,15,0,23,4,31
210429,210429,2021-12-31 23:30:00,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,30,0,23,4,31


In [115]:
day_list = []
energy_df['Date'] = pd.to_datetime(energy_df['Date'])
for time in energy_df['Date']:
    day = time.weekday()
    day_list.append(day)
energy_df['Day of Week'] = day_list

In [116]:
energy_df

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,minute_interval,Season,Hour of day,Day of Week,Day
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,15,0,0,4,1
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,30,0,0,4,1
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,45,0,0,4,1
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,0,1,4,1
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,15,0,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,210426,2021-12-31 22:45:00,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,45,0,22,4,31
210427,210427,2021-12-31 23:00:00,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,0,23,4,31
210428,210428,2021-12-31 23:15:00,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,15,0,23,4,31
210429,210429,2021-12-31 23:30:00,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,30,0,23,4,31


In [117]:
energy_df['Is Weekend'] = [day == 5 or day == 6 for day in energy_df['Day of Week']]

In [118]:
energy_df

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,minute_interval,Season,Hour of day,Day of Week,Day,Is Weekend
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,15,0,0,4,1,False
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,30,0,0,4,1,False
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,45,0,0,4,1,False
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,0,1,4,1,False
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,15,0,1,4,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,210426,2021-12-31 22:45:00,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,45,0,22,4,31,False
210427,210427,2021-12-31 23:00:00,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,0,23,4,31,False
210428,210428,2021-12-31 23:15:00,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,15,0,23,4,31,False
210429,210429,2021-12-31 23:30:00,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,30,0,23,4,31,False


In [119]:
clean_weekend = []
for b in energy_df['Is Weekend']:
    if b==True:
        clean_weekend.append(1)
    elif b==False:
        clean_weekend.append(0)
energy_df['Is Weekend Clean'] = clean_weekend

In [120]:
energy_df

,Unnamed: 0,Date,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,minute_interval,Season,Hour of day,Day of Week,Day,Is Weekend,Is Weekend Clean
0,0,2016-01-01 00:15:00,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,15,0,0,4,1,False,0
1,1,2016-01-01 00:30:00,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,30,0,0,4,1,False,0
2,2,2016-01-01 00:45:00,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,45,0,0,4,1,False,0
3,3,2016-01-01 01:00:00,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,0,1,4,1,False,0
4,4,2016-01-01 01:15:00,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,15,0,1,4,1,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,210426,2021-12-31 22:45:00,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,45,0,22,4,31,False,0
210427,210427,2021-12-31 23:00:00,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,0,23,4,31,False,0
210428,210428,2021-12-31 23:15:00,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,15,0,23,4,31,False,0
210429,210429,2021-12-31 23:30:00,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,30,0,23,4,31,False,0


In [121]:
energy_df = energy_df.drop(columns=['Is Weekend'])

In [123]:
energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210431 entries, 0 to 210430
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Unnamed: 0         210431 non-null  int64         
 1   Date               210431 non-null  datetime64[ns]
 2   Haryana Load (MW)  210431 non-null  float64       
 3   temp               210431 non-null  float64       
 4   dew                210431 non-null  float64       
 5   humidity           210431 non-null  float64       
 6   windspeed          210431 non-null  float64       
 7   winddir            210431 non-null  float64       
 8   cloudcover         210431 non-null  float64       
 9   visibility         210431 non-null  float64       
 10  month              210431 non-null  int64         
 11  minute_interval    210431 non-null  int64         
 12  Season             210431 non-null  int64         
 13  Hour of day        210431 non-null  int64   

In [127]:
energy_df.drop(columns='Date', inplace=True)

In [130]:
energy_df.drop(columns='Unnamed: 0', inplace=True)

In [131]:
energy_df.corr()

,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,minute_interval,Season,Hour of day,Day of Week,Day,Is Weekend Clean
Haryana Load (MW),1.000000,0.504668,0.617157,0.016372,0.087153,-0.029084,2.879013e-01,0.325990,0.177620,1.562659e-03,0.259011,0.114884,-0.041774,0.006339,-0.056105
temp,0.504668,1.000000,0.538345,-0.588910,0.362062,0.134242,2.392645e-01,0.692226,0.054718,-1.113263e-05,0.363203,0.219212,-0.005567,-0.011352,-0.007670
dew,0.617157,0.538345,1.000000,0.327262,0.073790,-0.165958,5.543215e-01,0.303009,0.237475,-8.714396e-06,0.462137,-0.013864,0.005373,-0.018056,-0.000827
humidity,0.016372,-0.588910,0.327262,1.000000,-0.327792,-0.303280,2.840298e-01,-0.499458,0.140681,3.719440e-06,0.006129,-0.280852,0.010816,0.005880,0.007714
windspeed,0.087153,0.362062,0.073790,-0.327792,1.000000,0.483971,1.795850e-01,0.345374,-0.164791,-8.582570e-06,-0.082914,0.141720,0.014333,0.015660,0.012012
winddir,-0.029084,0.134242,-0.165958,-0.303280,0.483971,1.000000,-8.420179e-02,0.134194,-0.113523,-8.798599e-06,-0.100639,0.134540,-0.006979,0.004788,0.002149
cloudcover,0.287901,0.239265,0.554322,0.284030,0.179585,-0.084202,1.000000e+00,0.117772,-0.000521,-9.680602e-07,0.096489,0.018671,-0.005450,0.002120,-0.012627
visibility,0.325990,0.692226,0.303009,-0.499458,0.345374,0.134194,1.177724e-01,1.000000,-0.016544,-9.025046e-06,0.225875,0.191557,-0.005567,0.009306,0.003059
month,0.177620,0.054718,0.237475,0.140681,-0.164791,-0.113523,-5.208113e-04,-0.016544,1.000000,-1.020770e-05,0.579282,-0.000013,0.001226,0.010054,0.000959
minute_interval,0.001563,-0.000011,-0.000009,0.000004,-0.000009,-0.000009,-9.680602e-07,-0.000009,-0.000010,1.000000e+00,-0.000008,-0.000011,0.000003,-0.000011,-0.000004


In [132]:
energy_df

,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,minute_interval,Season,Hour of day,Day of Week,Day,Is Weekend Clean
0,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,15,0,0,4,1,0
1,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,30,0,0,4,1,0
2,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,45,0,0,4,1,0
3,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,0,1,4,1,0
4,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,15,0,1,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,45,0,22,4,31,0
210427,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,0,23,4,31,0
210428,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,15,0,23,4,31,0
210429,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,30,0,23,4,31,0


In [133]:
# energy_df.to_csv('Complete_Dataset_added_Calendar_features_2.csv')

In [134]:
energy_df

,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,minute_interval,Season,Hour of day,Day of Week,Day,Is Weekend Clean
0,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,15,0,0,4,1,0
1,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,30,0,0,4,1,0
2,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,45,0,0,4,1,0
3,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,0,1,4,1,0
4,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,15,0,1,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,45,0,22,4,31,0
210427,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,0,23,4,31,0
210428,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,15,0,23,4,31,0
210429,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,30,0,23,4,31,0


In [135]:
df_train = energy_df.iloc[:int(energy_df.shape[0]*0.7)]

In [136]:
df_train

,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,minute_interval,Season,Hour of day,Day of Week,Day,Is Weekend Clean
0,4099.37,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,15,0,0,4,1,0
1,3995.64,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,30,0,0,4,1,0
2,3954.80,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,45,0,0,4,1,0
3,3904.93,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,0,1,4,1,0
4,3654.68,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,15,0,1,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147296,4396.38,18.2,16.5,89.40,6.4,85.0,55.7,2.0,3,15,1,8,5,14,1
147297,4479.17,18.2,16.5,89.40,6.4,85.0,55.7,2.0,3,30,1,8,5,14,1
147298,4493.16,18.2,16.5,89.40,6.4,85.0,55.7,2.0,3,45,1,8,5,14,1
147299,4543.41,19.0,17.0,88.19,11.2,100.0,37.1,2.0,3,0,1,9,5,14,1


In [137]:
df_test = energy_df[df_train.shape[0]:]

In [138]:
df_test

,Haryana Load (MW),temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,minute_interval,Season,Hour of day,Day of Week,Day,Is Weekend Clean
147301,4637.64,19.0,17.0,88.19,11.2,100.0,37.1,2.0,3,30,1,9,5,14,1
147302,4652.91,19.0,17.0,88.19,11.2,100.0,37.1,2.0,3,45,1,9,5,14,1
147303,4636.97,21.0,16.0,73.11,13.0,90.0,37.1,3.0,3,0,1,10,5,14,1
147304,4569.88,21.0,16.0,73.11,13.0,90.0,37.1,3.0,3,15,1,10,5,14,1
147305,4514.03,21.0,16.0,73.11,13.0,90.0,37.1,3.0,3,30,1,10,5,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,4712.55,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,45,0,22,4,31,0
210427,4649.58,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,0,23,4,31,0
210428,4571.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,15,0,23,4,31,0
210429,4510.39,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,30,0,23,4,31,0


In [34]:
# scaler = MinMaxScaler()

In [139]:
X = df_train.drop(columns='Haryana Load (MW)')

In [140]:
X

,temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,minute_interval,Season,Hour of day,Day of Week,Day,Is Weekend Clean
0,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,15,0,0,4,1,0
1,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,30,0,0,4,1,0
2,11.0,7.0,76.36,0.0,0.0,27.4,1.0,1,45,0,0,4,1,0
3,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,0,0,1,4,1,0
4,11.0,8.0,81.75,0.0,0.0,27.4,1.0,1,15,0,1,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147296,18.2,16.5,89.40,6.4,85.0,55.7,2.0,3,15,1,8,5,14,1
147297,18.2,16.5,89.40,6.4,85.0,55.7,2.0,3,30,1,8,5,14,1
147298,18.2,16.5,89.40,6.4,85.0,55.7,2.0,3,45,1,8,5,14,1
147299,19.0,17.0,88.19,11.2,100.0,37.1,2.0,3,0,1,9,5,14,1


In [141]:
y_train = energy_df['Haryana Load (MW)'][:df_train.shape[0]]

In [142]:
y_train

0         4099.37
1         3995.64
2         3954.80
3         3904.93
4         3654.68
           ...   
147296    4396.38
147297    4479.17
147298    4493.16
147299    4543.41
147300    4595.77
Name: Haryana Load (MW), Length: 147301, dtype: float64

In [143]:
# mmo = scaler.fit_transform(X, y_train)

In [144]:
# type(mmo)

In [145]:
# mmo.shape

In [42]:
# reg = ensemble.GradientBoostingRegressor()

In [43]:
# reg.fit(mmo, y_train)

GradientBoostingRegressor()

In [146]:
df_test.drop(columns='Haryana Load (MW)', inplace=True)

C:\Users\HP\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [147]:
df_test

,temp,dew,humidity,windspeed,winddir,cloudcover,visibility,month,minute_interval,Season,Hour of day,Day of Week,Day,Is Weekend Clean
147301,19.0,17.0,88.19,11.2,100.0,37.1,2.0,3,30,1,9,5,14,1
147302,19.0,17.0,88.19,11.2,100.0,37.1,2.0,3,45,1,9,5,14,1
147303,21.0,16.0,73.11,13.0,90.0,37.1,3.0,3,0,1,10,5,14,1
147304,21.0,16.0,73.11,13.0,90.0,37.1,3.0,3,15,1,10,5,14,1
147305,21.0,16.0,73.11,13.0,90.0,37.1,3.0,3,30,1,10,5,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210426,10.0,8.0,87.38,0.0,0.0,27.4,2.0,12,45,0,22,4,31,0
210427,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,0,0,23,4,31,0
210428,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,15,0,23,4,31,0
210429,9.2,7.3,88.04,0.0,0.0,28.5,1.3,12,30,0,23,4,31,0


In [148]:
# mpo = scaler.fit_transform(df_test)

In [49]:
# predictions = reg.predict(mpo)

In [58]:
# predictions = pd.DataFrame(predictions)

In [149]:
y_test = energy_df['Haryana Load (MW)'].iloc[df_train.shape[0]:]

In [150]:
y_test

147301    4637.64
147302    4652.91
147303    4636.97
147304    4569.88
147305    4514.03
           ...   
210426    4712.55
210427    4649.58
210428    4571.39
210429    4510.39
210430    4449.94
Name: Haryana Load (MW), Length: 63130, dtype: float64

In [151]:
import sklearn

In [152]:
# r2s = sklearn.metrics.r2_score(y_test, predictions[0])
# r2s

In [153]:
# params = {
#     "n_estimators": 1000,
#     "max_depth": 4,
#     "min_samples_split": 5,
#     "learning_rate": 0.1,
#     "loss": "squared_error",
# }

In [154]:
model = xgb.XGBRFRegressor()

In [155]:
model.fit(X, y_train)

XGBRFRegressor(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bytree=1,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='', max_bin=256,
               max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
               min_child_weight=1, missing=nan, monotone_constraints='()',
               n_estimators=100, n_jobs=0, num_parallel_tree=100,
               objective='reg:squarederror', predictor='auto', random_state=0,
               reg_alpha=0, sampling_method='uniform', scale_pos_weight=1, ...)

In [156]:
xgb_predictions = model.predict(df_test)

In [157]:
xgb_predictions

array([5050.036 , 5050.036 , 5050.036 , ..., 4486.8433, 4486.8433,
       4486.8433], dtype=float32)

In [158]:
r2s = sklearn.metrics.r2_score(y_test, xgb_predictions)
r2s

0.6150382054022898

In [172]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [173]:
rmse = np.sqrt(mean_squared_error(y_test, xgb_predictions))
rmse

1136.504824459056

In [174]:
mx = y_test.max()
mn = y_test.min()
nrmse = rmse/(mx - mn)

In [175]:
nrmse

0.10533238407546604